In [3]:
# Data processing
import pandas as pd

In [4]:
table1 = pd.read_csv("/mnt/c/ubuntu_ws/learning/data-analysis/BBC-analysis/data/BBC.csv")
first_column = table1.pop('User_id')
table1.insert(0, 'User_id', first_column)
table1['Date'] = pd.to_datetime(table1['Date'])
table1.head()

,User_id,Serviceid,Date
0,40709217,20,2018-01-02
1,40709217,20,2018-01-02
2,18246539,984,2018-03-22
3,18246539,269,2018-03-22
4,18246539,666,2018-04-01


In [5]:
table1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1733 entries, 0 to 1732
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   User_id    1733 non-null   int64         
 1   Serviceid  1733 non-null   int64         
 2   Date       1733 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(2)
memory usage: 40.7 KB


In [6]:
table1['RN'] = table1.groupby('User_id')['Date'].rank(method = 'first', ascending = True)
table1['NR'] = table1.groupby('User_id')['Date'].rank(method = 'first', ascending = False)

In [7]:
table1.head(10)

,User_id,Serviceid,Date,RN,NR
0,40709217,20,2018-01-02,1.0,1.0
1,40709217,20,2018-01-02,2.0,2.0
2,18246539,984,2018-03-22,1.0,3.0
3,18246539,269,2018-03-22,2.0,4.0
4,18246539,666,2018-04-01,3.0,1.0
5,18246539,326,2018-04-01,4.0,2.0
6,40725355,1014,2018-06-09,2.0,5.0
7,40725355,1014,2018-06-09,3.0,6.0
8,40725355,299,2018-06-08,1.0,7.0
9,40725355,1014,2018-06-12,4.0,3.0


In [8]:
# making boolean series for a team name
first = (table1["RN"] == 1)
second = (table1["RN"] == 2)
last = (table1["NR"] == 1)


firsttable = table1.copy()
secondtable = table1.copy()
lasttable = table1.copy()

In [9]:
# filtering data
FirstRank = firsttable.where(first, inplace = False)
FirstRank = FirstRank.rename(columns={"Serviceid":"FirstServiceid", "Date":"FirstServiceDate"}).dropna().drop(["RN","NR"],axis = True)

SecondRank = secondtable.where(second, inplace = False)
SecondRank = SecondRank.rename(columns={"Serviceid":"SecondServiceid", "Date":"SecondServiceDate"}).dropna().drop(["RN","NR"],axis = True)

LastRank = lasttable.where(last, inplace = False)
LastRank = LastRank.rename(columns={"Serviceid":"LastServiceid", "Date":"LastServiceDate"}).dropna().drop(["RN","NR"],axis = True)

In [10]:
FirstLast = pd.merge(pd.merge(FirstRank,SecondRank,on='User_id'),LastRank,on='User_id')
FirstLast.head()

,User_id,FirstServiceid,FirstServiceDate,SecondServiceid,SecondServiceDate,LastServiceid,LastServiceDate
0,40709217.0,20.0,2018-01-02,20.0,2018-01-02,20.0,2018-01-02
1,18246539.0,984.0,2018-03-22,269.0,2018-03-22,666.0,2018-04-01
2,40725355.0,299.0,2018-06-08,1014.0,2018-06-09,983.0,2018-06-25
3,40727154.0,487.0,2018-01-04,742.0,2018-01-04,487.0,2018-01-06
4,40716114.0,1136.0,2018-01-02,1136.0,2018-01-02,1136.0,2018-01-02


In [11]:
count = table1.groupby('User_id')['Serviceid'].nunique()
count = pd.DataFrame(count).reset_index(drop=False).rename(columns={"Serviceid":"TotalService"})

In [12]:
BBC_FirstLast = pd.merge(count,FirstLast, on = 'User_id')
BBC_FirstLast.head()

,User_id,TotalService,FirstServiceid,FirstServiceDate,SecondServiceid,SecondServiceDate,LastServiceid,LastServiceDate
0,2464231,10,18.0,2018-01-07,667.0,2018-01-16,982.0,2018-07-23
1,3676235,9,273.0,2018-01-05,481.0,2018-01-06,355.0,2018-04-12
2,4958104,7,946.0,2018-01-22,310.0,2018-01-22,268.0,2018-06-30
3,11642209,14,20.0,2018-01-01,666.0,2018-01-02,666.0,2018-06-29
4,18246539,4,984.0,2018-03-22,269.0,2018-03-22,666.0,2018-04-01


In [13]:
BBC_FirstLast.to_csv('/mnt/c/ubuntu_ws/learning/data-analysis/BBC-analysis/data/BBC_FirstLast.csv')